# Fáza 1

**Michaela Gubovská a Jakub Hajdu**

In [ ]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.stats.api as sms

## Základný opis dát a ich charakteristika - dáta zo súboru profiles.csv

Najskôr si načítame dáta z "profiles.csv" do data frame.

In [ ]:
filename_p = "data/profiles.csv"
dfp = pd.read_csv(filename_p, sep='\t')
## separátorom dát je znak tabulátora

Počet záznamov:

In [ ]:
len(dfp)

Počet atribútov:

In [ ]:
len(dfp.columns.values)

Jednotlivé atribúty a ich typy:

In [ ]:
dfp.dtypes

In [ ]:
dfp[['blood_group', 'race', 'sex']].describe()

In [ ]:
sns.histplot(data=dfp['blood_group'])

In [ ]:
dfp.groupby(['blood_group', 'sex']).size()

In [ ]:
sns.histplot(data=dfp['race'])

In [ ]:
sns.histplot(data=dfp['sex'])


Overíme, či sa v dátach nevyskytujú chýbajúce hodnoty:

In [ ]:
dfp.isnull().sum()

Vypíšeme si začiatok dát. Už z výpisu atribútov bolo vidieť zvláštny stĺpec "Unnamed: 0".

In [ ]:
dfp.head()

## Identifikácia problémov dát zo súboru profiles.csv s navrhnutým riešením

Ako prvý problém vidíme nepotrebný index stĺpec s názvom "Unnamed: 0", ktorý môžme odstrániť.

In [ ]:
dfp.drop('Unnamed: 0', axis=1, inplace=True)
dfp.head()

Ďalší problém dát je nejednotný formát hodnôt v stĺpci "race":

In [ ]:
dfp.race.unique()

Po úprave sú hodnoty v konzistentnom formáte.

In [ ]:
dfp['race'] = dfp['race'].str.replace('white','White')
dfp['race'] = dfp['race'].str.replace('black','Black')
dfp['race'] = dfp['race'].str.replace('blsck','Black')
dfp.race.unique()

Ako ďalší problém si môžeme všimnúť rôzne formáty dátumov v stĺpci "birthdate". Tieto dátumy preto upravíme na jednotný formát yy-mm-dd.

In [ ]:
dfp['birthdate'] = pd.to_datetime(dfp.birthdate)

dfp['birthdate'] = dfp['birthdate'].dt.strftime('%Y-%m-%d')
dfp.head()

## Základný opis dát a ich charakteristika - dáta zo súboru labor.csv

In [ ]:
filename_l = "data/labor.csv"
dfl = pd.read_csv(filename_l, sep='\t')
dfl.head()

Počet záznamov:

In [ ]:
len(dfl)

Počet atribútov:

In [ ]:
len(dfl.columns.values)

Jednotlivé atribúty a ich typy:

In [ ]:
dfl.dtypes

In [ ]:
dfl[['weight', 'leukocyty', 'hemoglobin', 'trombocyty', 'alt', 'ast', 'alp', 'hematokrit', 'hbver', 'etytr', 
     'er-cv', 'erytrocyty']].describe()

In [ ]:
sns.histplot(data=dfl.hemoglobin)

In [ ]:
sns.histplot(data=dfl.hematokrit)

In [ ]:
sns.histplot(data=dfl.erytrocyty)

In [ ]:
sns.histplot(data=dfl.leukocyty)

In [ ]:
sns.histplot(data=dfl.trombocyty)

## Identifikácia problémov dát zo súboru labor.csv s navrhnutým riešením

Ako prvý problém znova vidíme nepotrebný index stĺpec s názvom "Unnamed: 0", ktorý môžeme odstrániť.

In [ ]:
dfl.drop('Unnamed: 0', axis=1, inplace=True)
dfl.head()

V dátach sa nachádzajú duplicitné záznamy.

In [ ]:
len(dfl) - len(dfl.drop_duplicates()) ## 99 duplicitnych zaznamov

In [ ]:
dfl = dfl.drop_duplicates() ## odstranene duplicitne zaznamy
len(dfl)

Identifikovali sme problém s dátami vo forme zlých hodnôt (prípadne rozdelenia hodnôt) hmotnosti. Možným riešením by bolo odstránenie záznamov s hmotnosťou <= 0, keďže takýchto záznamov je z celkového počtu približne 2,3%.

In [ ]:
len(dfl[dfl.weight <= 0]) / len(dfl) * 100

In [ ]:
sns.histplot(data=dfl['weight'])

Takisto sa v dátach vyskytujú chýbajúce hodnoty. Vzhľadom na ich relatívne nízku početnosť v porovnaní s celkovým počtom záznamov by možným riešením bolo nahradenie chýbajúcich hodnôt priemernými hodnotami daného atribútu (berúc do úvahy zistenie, že chýbajúce hodnoty sa vyskytujú len a práve pri číselných atribútoch, ktoré sú výsledkom samotného vyšetrenia/merania).

In [ ]:
dfl.isnull().sum()

In [ ]:
len(dfl[dfl.isnull().any(axis=1)])

Ďalším problémom je nekonzistentné vyplnenie stĺpca smoker. Tento problém už teraz vieme jednoducho opraviť.

In [ ]:
dfl.smoker.unique()

In [ ]:
dfl['smoker'] = dfl['smoker'].str.replace('N','no')
dfl['smoker'] = dfl['smoker'].str.replace('Y','yes')
dfl.smoker.unique()

## Párová analýza dát

Pre rýchle približné zistenie závislostí medzi dvojicami atribútov si najskôr vykreslíme pairplot všetkých z nich.

In [ ]:
sns.pairplot(dfl[['weight', 'leukocyty', 'hemoglobin', 'trombocyty', 'alt', 'ast', 'alp', 'hematokrit', 'hbver', 'etytr', 
     'er-cv', 'erytrocyty']])

Väčšina z dvojíc atribútov medzi sebou nemá závislosť a ich grafy vyzerajú podobne ako tento príklad:

In [ ]:
sns.pairplot(dfl[['hemoglobin', 'trombocyty']])

Dvojica, ktorá sa podľa grafu vizuálne najviac približuje závislosti, je *alt* (alanín transmináza) a *erytrocyty*. S vyššou hodnotou erytrocytov sa zvyšuje aj hodnota alt. Krivka však nemá príliš blízko k tvaru priamky y=x, pripomína skôr exponenciálnu funkciu (resp. logaritmickú, podľa zvolenia osí).

In [ ]:
sns.pairplot(dfl[['erytrocyty', 'alt']])

**Závislosti jednotlivých atribútov od predikovanej premennej**

Predikovaná premenná (indikátor) nám dáta rozdeľuje na dve skupiny (hodota indikátora 0 a 1). Pomocou grafov si môžeme vykresľovať distribúcie jednotlibých premenných rozdelené podľa indikátora a následne tieto dve skupiny porovnávať. Pri väčšine atribútov sme nepozorovali takmer žiaden rozdiel v závislosti od hodnoty indikátora, ako vidíme na nasledujúcom príklade:

In [ ]:
sns.boxplot(data=[dfl[dfl['indicator'] == 0]['ast'], dfl[dfl['indicator'] == 1]['ast']])

Tri atribúty v grafoch vizuálne preukázali rozdiel vzhľadom na indikátor: *hemoglobín*, *alp (alkalická fosfatáza)* a *hematokrit*. Aby bol rozdiel lepšie viditeľný, vykreslili sme závislosť týchto atribútov od indikátora okrem box-plotu aj cez joint-plot a kde-plot.

In [ ]:
sns.boxplot(data=[dfl[dfl['indicator'] == 0]['hemoglobin'], dfl[dfl['indicator'] == 1]['hemoglobin']])

In [ ]:
sns.jointplot(x='indicator', y='hemoglobin', data=dfl).plot_joint(sns.kdeplot, n_levels=10)

Z grafu môžeme pozorovať hustejší výskyt vyšších hodnôt hemoglobínu pri indikátore s hodnotou 1. Hodnoty hemoglobínu sú výrazne nižšie v prípade indikátora s hodnotou 0. 

In [ ]:
sns.boxplot(data=[dfl[dfl['indicator'] == 0]['alp'], dfl[dfl['indicator'] == 1]['alp']])

In [ ]:
sns.jointplot(x='indicator', y='alp', data=dfl).plot_joint(sns.kdeplot, n_levels=10)

Z grafu môžeme pozorovať výrazne väčší výskyt vysokých hodnôt alp pri indikátore s hodnotou 1.

In [ ]:
sns.boxplot(data=[dfl[dfl['indicator'] == 0]['hematokrit'], dfl[dfl['indicator'] == 1]['hematokrit']])

In [ ]:
sns.jointplot(x='indicator', y='hematokrit', data=dfl).plot_joint(sns.kdeplot, n_levels=10)

Z grafu môžeme pozorovať výrazne väčší počet nízkych hodnôt hematokritu ak je hodnota indikátora rovná 1. 

## Formulácia a štatistické overenie hypotéz o dátach

**Prvá hypotéza**

**Chceme overiť, či má hladina hemoglobínu vplyv na indikátor.**

Určíme si naše hypotézy nasledovne:

**$H_0$ (nulová hypotéza)**: Hladina hemoglobínu pacientov s indikátorom 0 **je** v priemere **rovnaká** ako hladina hemoglobínu pacientov s indikátorom 1.

**$H_1 = H_A$ (alternatívna hypotéza)**: Hladina hemoglobínu pacientov s indikátorom 0 **je** v priemere **iná/väčšia/menšia** ako hladina hemoglobínu pacientov s indikátorom 1.

Rozdelíme si hladiny hemoglobínu na dve skupiny podľa indikátora (odstránime NaN, ktorých bolo dokopy len 30).

In [ ]:
hemo_0 = dfl[dfl['indicator'] == 0]['hemoglobin'].dropna()
hemo_1 = dfl[dfl['indicator'] == 1]['hemoglobin'].dropna()

In [ ]:
sns.boxplot(data=[hemo_0, hemo_1])

In [ ]:
d = {'0': hemo_0, '1': hemo_1}
df = pd.DataFrame(data=d)
sns.kdeplot(data=df, fill = True, alpha = 0.5)

Z prvotných grafov sa vizuálne zdá, že rozdiel existuje. Musíme však zistiť, či je štatisticky signifikantný. Na to sa často používa **Studentov t-test**, pre ktorý ale musia skupiny dát spĺňať určité podmienky. Prvou je, že obe skupiny musia pochádzať z normálneho rozdelenia.

In [ ]:
sns.histplot(data=hemo_0)

In [ ]:
sns.histplot(data=hemo_1)

Prvotne sa môže zdať, že obe supiny majú normálne rozdelenie. Tento predpoklad si však musíme lepšie overiť.

In [ ]:
##funkcia na detekciu outlierov
def identify_outliers(a):
    lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
    upper = a.quantile(0.75) + 1.5 * stats.iqr(a)
    
    return a[(a > upper) | (a < lower)]

In [ ]:
hemo_0_out = identify_outliers(hemo_0)
hemo_1_out = identify_outliers(hemo_1)
print(len(hemo_0_out))
print(len(hemo_1_out))

Obe skupiny obsahujú outlierov, preto ich odstránime:

In [ ]:
hemo_0 = hemo_0.drop(hemo_0_out.index)
hemo_1 = hemo_1.drop(hemo_1_out.index)

In [ ]:
_ = sm.ProbPlot(hemo_0, fit=True).qqplot(line='45')

In [ ]:
_ = sm.ProbPlot(hemo_1, fit=True).qqplot(line='45')

Z QQ-plotu sa javí, že skupiny nepochádzajú z normálneho rozdelenia, ale ich rozdelenia sa na seba podobajú. 

In [ ]:
stats.kurtosis(hemo_0)

In [ ]:
stats.kurtosis(hemo_1)

In [ ]:
stats.skew(hemo_0)

In [ ]:
stats.skew(hemo_1)

Rozdelenia našich skupín majú podobné vlastnosti asymetrie (skew) aj špičatosti (kurtosis).

Či naozaj nepochádzajú z normálneho rozdelenia môžeme overiť **Shapiro-Wilkovým testom**.

In [ ]:
stats.shapiro(hemo_0)

In [ ]:
stats.shapiro(hemo_1)

Testovali sme nulovú hypotézu $H_0$, že dáta pochádzajú z normálneho rozdelenia. Ak je $p < 0,05$, nulovú hypotézu $H_0$ zamietame a dáta pravdepodobne pochádzajú z iného ako normálneho rozdelenia. Ak je $p > 0,05$, nulovú hypotézu $H_0$ nezamietame, teda na základe dát nemôžeme prehlásiť, že by dáta pochádzali z iného, ako normálneho rozdelenia.

Pre rozdelenia oboch skupín nám vyšla hodnota $p < 0,05$, nulovú hypotézu teda $H_0$ zamietame a dáta pravdepodobne pochádzajú z iného ako normálneho rozdelenia. Mali by sme teda použiť neparametrickú verziu t-testu, t. j. **Mann-Whitneyho U-test**

**Studentov t-test** vyžaduje tiež, aby rozdelenia oboch skupín mali podobnú varianciu. Môžme si ukázať tiež **Levenov test**, ktorý na overenie tejto podmienky slúži, aj keď už z predošlých testov je jasné, že Studentov t-test nemôžeme použiť. Levenov test testuje nulovú hypotézu $H_0$, že všetky vstupné vzorky pochádzajú z rozdelení s rovnakými varianciami. Ak $H_0$ nezamietame ($p > 0,05$), znamená to, že na základe dát nemôžeme prehlásiť, že by vzorky pochádzali z distribúcií s rôznymi varianciami.

In [ ]:
stats.levene(hemo_0, hemo_1)

p-value je blížiaci sa k nule, to znamená, že rozdelenia našich skupín nemajú ani podobné variancie.

Nebol teda splnený ani jeden z predpokladov na použitie t-testu, preto spustíme **Mann-Whitneyho U-test**.

In [ ]:
stats.mannwhitneyu(hemo_0, hemo_1)

Keďže $p < 0,001$, pravdepodobnosť chyby 1. rádu (že $H_0$ je pravdivá a my ju zamietame) je menej ako 1 promile. Našu nulovú hypotézu $H_0$ teda zamietame v prospech alternatívnej hypotézy $H_A$. Rozdiel v priemernej hladine hemoglobínu medzi pacientami s indikátorom 0 a indikátorom 1 je štatisticky signifikantný.

Môžeme si vizualizovať rozdiel medzi dvoma priemermi spolu s intervalmi spoľahlivosti, ktoré nám hovoria, že s N% pravdepodobnosťou (najčastejšie sa používa 95) sa skutočná hodnota priemeru bude nachádzať niekde v danom intervale.

In [ ]:
sms.DescrStatsW(hemo_0).tconfint_mean()

In [ ]:
sms.DescrStatsW(hemo_1).tconfint_mean()

In [ ]:
sns.barplot(x='indicator', y='hemoglobin', data=dfl[(dfl.indicator == 0) | (dfl.indicator == 1)].dropna(), 
            capsize=0.1, errwidth=2, palette=sns.color_palette("Greens"))

**Záver:** hypotézu $H_0$, že priemerná hladina hemoglobínu u pacientov s indikátorom 0 a pacientov s indikátorom 1 je rovnaká, sme zamietli v prospech alternatívnej hypotézy $H_A$. Priemerná hladina hemoglobínu u pacientov s indikátorom 0 je nižšia ako u pacientov s indikátorom 1 a tento rozdiel je štatisticky signifikantný.

**Druhá hypotéza**

**Chceme overiť, či má hodnota hematokritu vplyv na indikátor.**

Určíme si naše hypotézy nasledovne:

**$H_0$ (nulová hypotéza)**: Hodnota hematokritu pacientov s indikátorom 0 **je** v priemere **rovnaká** ako hodnota hematokritu pacientov s indikátorom 1.

**$H_1 = H_A$ (alternatívna hypotéza)**: Hodnota hematokritu pacientov s indikátorom 0 **je** v priemere **iná/väčšia/menšia** ako hodnota hematokritu pacientov s indikátorom 1.

Rozdelíme si hodnoty hematokritu na dve skupiny podľa indikátora (odstránime NaN, ktorých bolo dokopy len 30).

In [ ]:
hema_0 = dfl[dfl['indicator'] == 0]['hematokrit'].dropna()
hema_1 = dfl[dfl['indicator'] == 1]['hematokrit'].dropna()

In [ ]:
sns.boxplot(data=[hema_0, hema_1])

Z prvotného grafu sa vizuálne zdá, že rozdiel existuje. Musíme však zistiť, či je štatisticky signifikantný. Na to sa často používa **Studentov t-test**, pre ktorý ale musia skupiny dát spĺňať určité podmienky. Prvou je, že obe skupiny musia pochádzať z normálneho rozdelenia.

In [ ]:
sns.histplot(data=hema_0)

In [ ]:
sns.histplot(data=hema_1)

Prvotne sa môže zdať, že skupina s indikátorom 0 má normálne rozdelenie a skupina s indikátorom 1 nie je z normálneho rozdelenia. Tento predpoklad si však musíme lepšie overiť.

In [ ]:
hema_0_out = identify_outliers(hema_0)
hema_1_out = identify_outliers(hema_1)
print(len(hema_0_out))
print(len(hema_1_out))

Obe skupiny obsahujú outlierov, preto ich odstránime:

In [ ]:
hema_0 = hema_0.drop(hema_0_out.index)
hema_1 = hema_1.drop(hema_1_out.index)

In [ ]:
_ = sm.ProbPlot(hema_0, fit=True).qqplot(line='45')

In [ ]:
_ = sm.ProbPlot(hema_1, fit=True).qqplot(line='45')

Z QQ-plotu sa javí, že skupiny nepochádzajú z normálneho rozdelenia, a aj ich rozdelenia sa od seba mierne líšia. 

In [ ]:
stats.kurtosis(hema_0)

In [ ]:
stats.kurtosis(hema_1)

In [ ]:
stats.skew(hema_0)

In [ ]:
stats.skew(hema_1)

Rozdelenia našich skupín majú podobné vlastnosti špičatosti (kurtosis), ale odlišné vlastnosti asymetrie (skew). Skupina s indikátorom 1 má rozdelenie s výraznejším zošikmením smerom doľava.

Či naozaj nepochádzajú z normálneho rozdelenia môžeme overiť **Shapiro-Wilkovým testom**.

In [ ]:
stats.shapiro(hema_0)

In [ ]:
stats.shapiro(hema_1)

Testovali sme nulovú hypotézu $H_0$, že dáta pochádzajú z normálneho rozdelenia. Ak je $p < 0,05$, nulovú hypotézu $H_0$ zamietame a dáta pravdepodobne pochádzajú z iného ako normálneho rozdelenia. Ak je $p > 0,05$, nulovú hypotézu $H_0$ nezamietame, teda na základe dát nemôžeme prehlásiť, že by dáta pochádzali z iného, ako normálneho rozdelenia.

Pre rozdelenia oboch skupín nám vyšla hodnota $p < 0,05$, nulovú hypotézu teda $H_0$ zamietame a dáta pravdepodobne pochádzajú z iného ako normálneho rozdelenia. Mali by sme teda použiť neparametrickú verziu t-testu, t. j. **Mann-Whitneyho U-test**

**Studentov t-test** vyžaduje tiež, aby rozdelenia oboch skupín mali podobnú varianciu. Môžme si ukázať tiež **Levenov test**, ktorý na overenie tejto podmienky slúži, aj keď už z predošlých testov je jasné, že Studentov t-test nemôžeme použiť. Levenov test testuje nulovú hypotézu $H_0$, že všetky vstupné vzorky pochádzajú z rozdelení s rovnakými varianciami. Ak $H_0$ nezamietame ($p > 0,05$), znamená to, že na základe dát nemôžeme prehlásiť, že by vzorky pochádzali z distribúcií s rôznymi varianciami.

In [ ]:
stats.levene(hema_0, hema_1)

p-value je blížiaci sa k nule, to znamená, že rozdelenia našich skupín nemajú ani podobné variancie.

Nebol teda splnený ani jeden z predpokladov na použitie t-testu, preto spustíme **Mann-Whitneyho U-test**.

In [ ]:
stats.mannwhitneyu(hema_0, hema_1)

Keďže $p < 0,001$, pravdepodobnosť chyby 1. rádu (že $H_0$ je pravdivá a my ju zamietame) je menej ako 1 promile. Našu nulovú hypotézu $H_0$ teda zamietame v prospech alternatívnej hypotézy $H_A$. Rozdiel v priemernej hodnote hematokritu medzi pacientami s indikátorom 0 a indikátorom 1 je štatisticky signifikantný.

Môžeme si vizualizovať rozdiel medzi dvoma priemermi spolu s intervalmi spoľahlivosti, ktoré nám hovoria, že s N% pravdepodobnosťou (najčastejšie sa používa 95) sa skutočná hodnota priemeru bude nachádzať niekde v danom intervale.

In [ ]:
sms.DescrStatsW(hema_0).tconfint_mean()

In [ ]:
sms.DescrStatsW(hema_1).tconfint_mean()

In [ ]:
sns.barplot(x='indicator', y='hematokrit', data=dfl[(dfl.indicator == 0) | (dfl.indicator == 1)].dropna(), 
            capsize=0.1, errwidth=2, palette=sns.color_palette("Blues"))

**Záver:** hypotézu $H_0$, že priemerná hodnota hematokritu u pacientov s indikátorom 0 a pacientov s indikátorom 1 je rovnaká, sme zamietli v prospech alternatívnej hypotézy $H_A$. Priemerná hodnota hematokritu u pacientov s indikátorom 0 je vyššia ako u pacientov s indikátorom 1 a tento rozdiel je štatisticky signifikantný.